# Create and store embeddings in vector database

Current limitation: store them sorted by date. We want a reference later which parts of the journal were used to generate a response and when the user wrote this parts in his journal.


TODO: 
- test with whole journal, are entries similar to other version?
- why do vectors have another format then with OpenAI directly? (1024 vs 1536)
- why didn't I need OpenAI API-Key first hand for embeddings? parallel version of jupyter running?

In [103]:
import os
import numpy as np
import pandas as pd
import pinecone
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI

In [60]:
text = f"""
2023.04.13
Heute ist ein anstrengender Tag. Ich fühle mich seit ein paar Tagen sehr gestresst und kann kaum schlafen. In der Nacht wache ich ständig auf und meine Gedanken kreisen ununterbrochen um all die Dinge, die ich noch erledigen muss. Ich habe das Gefühl, dass ich keine Energie mehr habe und bin deshalb heute den ganzen Tag müde und unkonzentriert. 
Ich hoffe, dass sich das bald ändert und ich wieder zu meiner alten Energie zurückfinde.

2023.04.14
Heute geht es mir schon etwas besser. Ich habe gestern Abend ein paar Entspannungsübungen gemacht und konnte dadurch besser schlafen. Meine Gedanken sind immer noch etwas chaotisch, aber ich habe das Gefühl, dass ich langsam wieder Kontrolle darüber bekomme. 
Ich habe heute auch schon ein paar Dinge von meiner To-Do-Liste abhaken können, was mir ein gutes Gefühl gibt.

2023.04.15
Ich bin wirklich stolz auf mich, denn ich habe heute schon sehr viel geschafft. Ich fühle mich energiegeladen und produktiv. Die Entspannungsübungen scheinen zu helfen und ich kann meine Gedanken besser sortieren. Ich habe sogar schon anfangen können, an einem neuen Projekt zu arbeiten, auf das ich mich schon seit Wochen freue. Es fühlt sich gut an, wieder in die richtige Richtung zu gehen.

2023.04.16
Ich bin so froh, dass ich die letzten Tage so viel Energie hatte. Es hat mir geholfen, die Dinge, die ich schon lange vor mir hergeschoben habe, endlich anzugehen. Heute habe ich fast alles von meiner To-Do-Liste erledigt und fühle mich unglaublich zufrieden. Ich habe das Gefühl, dass ich meine alte Kraft zurückgewonnen habe und bin optimistisch für die Zukunft.

2023.04.17
Heute ist ein guter Tag. Ich fühle mich ausgeglichen und glücklich. Die letzten Tage haben mir gezeigt, dass ich auch in schwierigen Situationen durchhalten kann. 
Ich habe gelernt, dass es wichtig ist, auf mich selbst zu achten und mir Zeit für Entspannung und Regeneration zu nehmen. Ich bin dankbar für alles, was ich erreicht habe und freue mich auf das, was noch kommt.
"""

In [61]:
def split_text(content):
    """Split content into entries based on date pattern.
       content: string with journal entries"""
    # Define a regular expression pattern to match dates
    date_pattern = r'\d{4}.\d{2}.\d{2}'

    # Split the content based on the date pattern
    entries = re.split(date_pattern, content)

    # Extract dates from the content
    dates = re.findall(date_pattern, content)

    # Create a dictionary with dates and corresponding entries
    data = {'Date': dates, 'Text': [entry.strip() for entry in entries[1:]]}

    # zip(dates, entries[1:])
    return entries[1:]

In [62]:
entries = split_text(text)

In [63]:
entries[0]

'\nHeute ist ein anstrengender Tag. Ich fühle mich seit ein paar Tagen sehr gestresst und kann kaum schlafen. In der Nacht wache ich ständig auf und meine Gedanken kreisen ununterbrochen um all die Dinge, die ich noch erledigen muss. Ich habe das Gefühl, dass ich keine Energie mehr habe und bin deshalb heute den ganzen Tag müde und unkonzentriert. \nIch hoffe, dass sich das bald ändert und ich wieder zu meiner alten Energie zurückfinde.\n\n'

In [64]:
# initialize OpenAI Embeddings Model + create embeddings
embeddings = OpenAIEmbeddings(model_name="ada")

In [65]:
# initialize Pinecone
pinecone.init(
    api_key="721b7475-1514-4e43-b533-6e92368d009b", 
    environment="asia-southeast1-gcp-free")
index_name = "aicoach"

In [67]:
# function to split entries into chunks using langchain
def langchain_textsplitter(content_entry):
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 100,
        chunk_overlap = 20,
    )
    
    return text_splitter.create_documents([content_entry])

In [100]:
# for each entry create chunk, append list

chunks = []
for entry in entries:
    chunks.append(langchain_textsplitter(entry))

In [79]:
chunks[0][0].page_content # extract plane text, 1st chunk of 1st entry

'Heute ist ein anstrengender Tag. Ich fühle mich seit ein paar Tagen sehr gestresst und kann kaum'

In [82]:
# create embedding for this chunk to test
query_result = embeddings.embed_query(chunks[0][0].page_content)
query_result

[0.016200980083297418,
 0.03753413612063291,
 0.032015010540111424,
 0.06309319253165013,
 0.0003347625297416978,
 0.04521203761502956,
 0.0035640117518149594,
 0.032259397032939695,
 0.042523752666306106,
 -0.02283004100103745,
 -0.07661607074719948,
 -0.0008381791949242503,
 0.032748177469176784,
 0.0381247436413334,
 0.025518324087115758,
 -0.025457227463908688,
 0.003734575331354017,
 -0.016628660171037167,
 0.007616802542883157,
 0.04362350678519442,
 0.04337911656707588,
 -0.010712400951734329,
 0.06480392033318967,
 -0.0165166500055984,
 0.04855202484501542,
 0.024764790996099663,
 0.00975520996650116,
 0.006751256958121872,
 0.029123067697170903,
 -0.040996320322323185,
 -0.019347492500041575,
 -0.032503787251058236,
 -0.0040502450208243545,
 0.05050713913938323,
 0.008645274629282615,
 0.015814030456814003,
 0.03820620828913634,
 -0.006797079891188457,
 -0.020803647002487206,
 -0.05706492035294083,
 -0.014469887982452282,
 -0.005017619552694177,
 0.00934789417806701,
 -0.00608

In [101]:
# upload embeddings to pinecone
for i in range(len(chunks)):
    search = Pinecone.from_documents(chunks[i], embeddings, index_name=index_name)
    
# list comprehension doesn't work. Why?
# search = Pinecone.from_documents([c for c in chunks], embeddings, index_name=index_name)

In [46]:
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [99]:
query = "Why is the author feeling awesome?"
result = search.similarity_search(query)
result

[Document(page_content='was mir ein gutes Gefühl gibt.', metadata={}),
 Document(page_content='und bin optimistisch für die Zukunft.', metadata={}),
 Document(page_content='erreicht habe und freue mich auf das, was noch kommt.', metadata={}),
 Document(page_content='freue. Es fühlt sich gut an, wieder in die richtige Richtung zu gehen.', metadata={})]

In [105]:
API_0 = os.getenv('OPENAI_API_KEY')
GPT_MODEL = "gpt-3.5-turbo"

In [118]:
llm = OpenAI(temperature = 0, model_name=GPT_MODEL)

In [121]:
prompt = f"""
Take the users query and answer it by including what the user has written
in his journal.
query: '{query}'
journal parts: '{[c.page_content for c in result]}'

"""

In [122]:
print(llm(prompt))

According to the journal parts, the author is feeling awesome because they have achieved something and are optimistic about the future. They also mention feeling good about going in the right direction. It seems that the author's accomplishments and positive outlook are giving them a good feeling.
